In [1]:
import pandas as pd
import matplotlib as plt
import matplotlib.cm as cm
import numpy as np

# Анализ данных

In [2]:
df = pd.read_csv("Training.csv")
df = df.drop(columns="Unnamed: 133")
df

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,muscle_wasting,vomiting,burning_micturition,spotting_ urination,fatigue,weight_gain,anxiety,cold_hands_and_feets,mood_swings,weight_loss,restlessness,lethargy,patches_in_throat,irregular_sugar_level,cough,high_fever,sunken_eyes,breathlessness,sweating,dehydration,indigestion,headache,yellowish_skin,dark_urine,nausea,loss_of_appetite,pain_behind_the_eyes,back_pain,constipation,abdominal_pain,...,internal_itching,toxic_look_(typhos),depression,irritability,muscle_pain,altered_sensorium,red_spots_over_body,belly_pain,abnormal_menstruation,dischromic _patches,watering_from_eyes,increased_appetite,polyuria,family_history,mucoid_sputum,rusty_sputum,lack_of_concentration,visual_disturbances,receiving_blood_transfusion,receiving_unsterile_injections,coma,stomach_bleeding,distention_of_abdomen,history_of_alcohol_consumption,fluid_overload.1,blood_in_sputum,prominent_veins_on_calf,palpitations,painful_walking,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
2,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
3,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
4,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Fungal infection
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4915,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,(vertigo) Paroymsal Positional Vertigo
4916,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,Acne
4917,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Urinary tract infection
4918,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,Psoriasis


In [3]:
column_names = df.columns.tolist()

In [4]:
columns_df = pd.DataFrame({
    'symptoms': column_names
})

columns_df.to_csv('symptoms.csv', index=False)

In [5]:
df['prognosis'].unique()

array(['Fungal infection', 'Allergy', 'GERD', 'Chronic cholestasis',
       'Drug Reaction', 'Peptic ulcer diseae', 'AIDS', 'Diabetes ',
       'Gastroenteritis', 'Bronchial Asthma', 'Hypertension ', 'Migraine',
       'Cervical spondylosis', 'Paralysis (brain hemorrhage)', 'Jaundice',
       'Malaria', 'Chicken pox', 'Dengue', 'Typhoid', 'hepatitis A',
       'Hepatitis B', 'Hepatitis C', 'Hepatitis D', 'Hepatitis E',
       'Alcoholic hepatitis', 'Tuberculosis', 'Common Cold', 'Pneumonia',
       'Dimorphic hemmorhoids(piles)', 'Heart attack', 'Varicose veins',
       'Hypothyroidism', 'Hyperthyroidism', 'Hypoglycemia',
       'Osteoarthristis', 'Arthritis',
       '(vertigo) Paroymsal  Positional Vertigo', 'Acne',
       'Urinary tract infection', 'Psoriasis', 'Impetigo'], dtype=object)

In [6]:
df['prognosis'].value_counts()

Fungal infection                           120
Hepatitis C                                120
Hepatitis E                                120
Alcoholic hepatitis                        120
Tuberculosis                               120
Common Cold                                120
Pneumonia                                  120
Dimorphic hemmorhoids(piles)               120
Heart attack                               120
Varicose veins                             120
Hypothyroidism                             120
Hyperthyroidism                            120
Hypoglycemia                               120
Osteoarthristis                            120
Arthritis                                  120
(vertigo) Paroymsal  Positional Vertigo    120
Acne                                       120
Urinary tract infection                    120
Psoriasis                                  120
Hepatitis D                                120
Hepatitis B                                120
Allergy      

# Модель

In [7]:
pip install catboost

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [8]:
if df.isnull().values.any():
    df = df.fillna(method='ffill')

X = df.drop(columns=['prognosis'])
y = df['prognosis']

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from catboost import CatBoostClassifier

model_c = CatBoostClassifier(iterations=500, learning_rate=0.01, depth=10)
model_c.fit(X_train, y_train, eval_set=(X_test, y_test), verbose=100)

0:	learn: 3.6277032	test: 3.6329970	best: 3.6329970 (0)	total: 475ms	remaining: 3m 57s
100:	learn: 1.0653302	test: 1.1091893	best: 1.1091893 (100)	total: 41.4s	remaining: 2m 43s
200:	learn: 0.5379541	test: 0.5741547	best: 0.5741547 (200)	total: 1m 22s	remaining: 2m 2s
300:	learn: 0.3050468	test: 0.3321408	best: 0.3321408 (300)	total: 2m 3s	remaining: 1m 21s
400:	learn: 0.1868163	test: 0.2075350	best: 0.2075350 (400)	total: 2m 45s	remaining: 40.8s
499:	learn: 0.1199458	test: 0.1355879	best: 0.1355879 (499)	total: 3m 26s	remaining: 0us

bestTest = 0.1355879121
bestIteration = 499



In [11]:
from sklearn.metrics import f1_score

y_pred = model_c.predict(X_test)
f1_weighted = f1_score(y_test, y_pred, average='weighted')
print(f"Взвешенный средний F1-скор: {f1_weighted:.2f}")

f1_per_class = f1_score(y_test, y_pred, average=None)
print("F1-скор для каждого класса:")

classes = df['prognosis'].unique()
for cls, score in zip(classes, f1_per_class):
    print(f"Класс {cls}: {score:.2f}")

Взвешенный средний F1-скор: 1.00
F1-скор для каждого класса:
Класс Fungal infection: 1.00
Класс Allergy: 1.00
Класс GERD: 1.00
Класс Chronic cholestasis: 1.00
Класс Drug Reaction: 1.00
Класс Peptic ulcer diseae: 1.00
Класс AIDS: 1.00
Класс Diabetes : 1.00
Класс Gastroenteritis: 1.00
Класс Bronchial Asthma: 1.00
Класс Hypertension : 1.00
Класс Migraine: 1.00
Класс Cervical spondylosis: 1.00
Класс Paralysis (brain hemorrhage): 1.00
Класс Jaundice: 1.00
Класс Malaria: 1.00
Класс Chicken pox: 1.00
Класс Dengue: 1.00
Класс Typhoid: 1.00
Класс hepatitis A: 1.00
Класс Hepatitis B: 1.00
Класс Hepatitis C: 0.98
Класс Hepatitis D: 0.98
Класс Hepatitis E: 1.00
Класс Alcoholic hepatitis: 1.00
Класс Tuberculosis: 1.00
Класс Common Cold: 1.00
Класс Pneumonia: 1.00
Класс Dimorphic hemmorhoids(piles): 1.00
Класс Heart attack: 1.00
Класс Varicose veins: 1.00
Класс Hypothyroidism: 1.00
Класс Hyperthyroidism: 1.00
Класс Hypoglycemia: 1.00
Класс Osteoarthristis: 1.00
Класс Arthritis: 1.00
Класс (vertigo) 

# Тестирование

In [12]:
ner = pd.read_csv("NER1.csv")
ner

,Word,Label
0,patient,O
1,:,O
2,my,O
3,mom,Subject
4,was,O
...,...,...
103,smoked,History
104,in,O
105,her,O
106,life,O


In [13]:
ner[ner['Label'] == 'Sign_symptom']

,Word,Label
21,felt,Sign_symptom
22,much,Sign_symptom
23,better,Sign_symptom
74,symptoms,Sign_symptom
77,coughing,Sign_symptom
80,fever,Sign_symptom


In [14]:
import pandas as pd
import torch
from transformers import BertModel, BertTokenizer
from sklearn.metrics.pairwise import cosine_similarity

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [15]:
symptoms_df = pd.read_csv('symptoms.csv')
symptoms_list = symptoms_df['symptoms'].tolist()

In [16]:
sign_symptom_words = ner[ner['Label'] == 'Sign_symptom']['Word'].tolist()
sign_symptom_embeddings = [get_embeddings(word) for word in sign_symptom_words]

In [17]:
symptoms_embeddings = [get_embeddings(symptom) for symptom in symptoms_list]

In [18]:
results_df = pd.DataFrame(0, index=np.arange(len(sign_symptom_words)), columns=symptoms_list)
results_df

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,muscle_wasting,vomiting,burning_micturition,spotting_ urination,fatigue,weight_gain,anxiety,cold_hands_and_feets,mood_swings,weight_loss,restlessness,lethargy,patches_in_throat,irregular_sugar_level,cough,high_fever,sunken_eyes,breathlessness,sweating,dehydration,indigestion,headache,yellowish_skin,dark_urine,nausea,loss_of_appetite,pain_behind_the_eyes,back_pain,constipation,abdominal_pain,...,internal_itching,toxic_look_(typhos),depression,irritability,muscle_pain,altered_sensorium,red_spots_over_body,belly_pain,abnormal_menstruation,dischromic _patches,watering_from_eyes,increased_appetite,polyuria,family_history,mucoid_sputum,rusty_sputum,lack_of_concentration,visual_disturbances,receiving_blood_transfusion,receiving_unsterile_injections,coma,stomach_bleeding,distention_of_abdomen,history_of_alcohol_consumption,fluid_overload.1,blood_in_sputum,prominent_veins_on_calf,palpitations,painful_walking,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
for i, (word, embedding) in enumerate(zip(sign_symptom_words, sign_symptom_embeddings)):
    for j, (symptom, sym_embedding) in enumerate(zip(symptoms_list, symptoms_embeddings)):
        similarity = cosine_similarity(embedding, sym_embedding)
        if similarity > 0.9:
            results_df.loc[i, symptom] = 1

In [20]:
results_df

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,muscle_wasting,vomiting,burning_micturition,spotting_ urination,fatigue,weight_gain,anxiety,cold_hands_and_feets,mood_swings,weight_loss,restlessness,lethargy,patches_in_throat,irregular_sugar_level,cough,high_fever,sunken_eyes,breathlessness,sweating,dehydration,indigestion,headache,yellowish_skin,dark_urine,nausea,loss_of_appetite,pain_behind_the_eyes,back_pain,constipation,abdominal_pain,...,internal_itching,toxic_look_(typhos),depression,irritability,muscle_pain,altered_sensorium,red_spots_over_body,belly_pain,abnormal_menstruation,dischromic _patches,watering_from_eyes,increased_appetite,polyuria,family_history,mucoid_sputum,rusty_sputum,lack_of_concentration,visual_disturbances,receiving_blood_transfusion,receiving_unsterile_injections,coma,stomach_bleeding,distention_of_abdomen,history_of_alcohol_consumption,fluid_overload.1,blood_in_sputum,prominent_veins_on_calf,palpitations,painful_walking,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [21]:
logical_or_result = results_df.any(axis=0).astype(int)
final_df = pd.DataFrame([logical_or_result], index=['Match Found'])
final_df

,itching,skin_rash,nodal_skin_eruptions,continuous_sneezing,shivering,chills,joint_pain,stomach_pain,acidity,ulcers_on_tongue,muscle_wasting,vomiting,burning_micturition,spotting_ urination,fatigue,weight_gain,anxiety,cold_hands_and_feets,mood_swings,weight_loss,restlessness,lethargy,patches_in_throat,irregular_sugar_level,cough,high_fever,sunken_eyes,breathlessness,sweating,dehydration,indigestion,headache,yellowish_skin,dark_urine,nausea,loss_of_appetite,pain_behind_the_eyes,back_pain,constipation,abdominal_pain,...,internal_itching,toxic_look_(typhos),depression,irritability,muscle_pain,altered_sensorium,red_spots_over_body,belly_pain,abnormal_menstruation,dischromic _patches,watering_from_eyes,increased_appetite,polyuria,family_history,mucoid_sputum,rusty_sputum,lack_of_concentration,visual_disturbances,receiving_blood_transfusion,receiving_unsterile_injections,coma,stomach_bleeding,distention_of_abdomen,history_of_alcohol_consumption,fluid_overload.1,blood_in_sputum,prominent_veins_on_calf,palpitations,painful_walking,pus_filled_pimples,blackheads,scurring,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze,prognosis
Match Found,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [22]:
y_pred_prob = model_c.predict_proba(final_df)

top_5_indices = np.argsort(y_pred_prob[0])[::-1][:5]
top_5_predictions = model_c.classes_[top_5_indices]
top_5_probabilities = y_pred_prob[0][top_5_indices]

for prediction, probability in zip(top_5_predictions, top_5_probabilities):
    print(f"Prediction: {prediction}, Probability: {probability:.4f}")

Prediction: Hypertension , Probability: 0.0885
Prediction: Allergy, Probability: 0.0758
Prediction: Migraine, Probability: 0.0686
Prediction: Paralysis (brain hemorrhage), Probability: 0.0683
Prediction: Heart attack, Probability: 0.0587


In [26]:
ner = pd.read_csv("NER2.csv")
sign_symptom_words = ner[ner['Label'] == 'Sign_symptom']['Word'].tolist()
sign_symptom_embeddings = [get_embeddings(word) for word in sign_symptom_words]

results_df = pd.DataFrame(0, index=np.arange(len(sign_symptom_words)), columns=symptoms_list)
for i, (word, embedding) in enumerate(zip(sign_symptom_words, sign_symptom_embeddings)):
    for j, (symptom, sym_embedding) in enumerate(zip(symptoms_list, symptoms_embeddings)):
        similarity = cosine_similarity(embedding, sym_embedding)
        if similarity > 0.9:
            results_df.loc[i, symptom] = 1

logical_or_result = results_df.any(axis=0).astype(int)
final_df = pd.DataFrame([logical_or_result], index=['Match Found'])

y_pred_prob = model_c.predict_proba(final_df)

top_5_indices = np.argsort(y_pred_prob[0])[::-1][:5]
top_5_predictions = model_c.classes_[top_5_indices]
top_5_probabilities = y_pred_prob[0][top_5_indices]

for prediction, probability in zip(top_5_predictions, top_5_probabilities):
    print(f"Prediction: {prediction}, Probability: {probability:.4f}")

Prediction: Hypertension , Probability: 0.0885
Prediction: Allergy, Probability: 0.0758
Prediction: Migraine, Probability: 0.0686
Prediction: Paralysis (brain hemorrhage), Probability: 0.0683
Prediction: Heart attack, Probability: 0.0587


In [24]:
ner = pd.read_csv("NER3.csv")
sign_symptom_words = ner[ner['Label'] == 'Sign_symptom']['Word'].tolist()
sign_symptom_embeddings = [get_embeddings(word) for word in sign_symptom_words]

results_df = pd.DataFrame(0, index=np.arange(len(sign_symptom_words)), columns=symptoms_list)
for i, (word, embedding) in enumerate(zip(sign_symptom_words, sign_symptom_embeddings)):
    for j, (symptom, sym_embedding) in enumerate(zip(symptoms_list, symptoms_embeddings)):
        similarity = cosine_similarity(embedding, sym_embedding)
        if similarity > 0.9:
            results_df.loc[i, symptom] = 1

logical_or_result = results_df.any(axis=0).astype(int)
final_df = pd.DataFrame([logical_or_result], index=['Match Found'])

y_pred_prob = model_c.predict_proba(final_df)

top_5_indices = np.argsort(y_pred_prob[0])[::-1][:5]
top_5_predictions = model_c.classes_[top_5_indices]
top_5_probabilities = y_pred_prob[0][top_5_indices]

for prediction, probability in zip(top_5_predictions, top_5_probabilities):
    print(f"Prediction: {prediction}, Probability: {probability:.4f}")

Prediction: (vertigo) Paroymsal  Positional Vertigo, Probability: 0.0859
Prediction: Allergy, Probability: 0.0469
Prediction: Hepatitis C, Probability: 0.0428
Prediction: Dimorphic hemmorhoids(piles), Probability: 0.0428
Prediction: Urinary tract infection, Probability: 0.0421


In [ ]:
ner = pd.read_csv("NER4.csv")
sign_symptom_words = ner[ner['Label'] == 'Sign_symptom']['Word'].tolist()
sign_symptom_embeddings = [get_embeddings(word) for word in sign_symptom_words]

results_df = pd.DataFrame(0, index=np.arange(len(sign_symptom_words)), columns=symptoms_list)
for i, (word, embedding) in enumerate(zip(sign_symptom_words, sign_symptom_embeddings)):
    for j, (symptom, sym_embedding) in enumerate(zip(symptoms_list, symptoms_embeddings)):
        similarity = cosine_similarity(embedding, sym_embedding)
        if similarity > 0.9:
            results_df.loc[i, symptom] = 1

logical_or_result = results_df.any(axis=0).astype(int)
final_df = pd.DataFrame([logical_or_result], index=['Match Found'])

y_pred_prob = model_c.predict_proba(final_df)

top_5_indices = np.argsort(y_pred_prob[0])[::-1][:5]
top_5_predictions = model_c.classes_[top_5_indices]
top_5_probabilities = y_pred_prob[0][top_5_indices]

for prediction, probability in zip(top_5_predictions, top_5_probabilities):
    print(f"Prediction: {prediction}, Probability: {probability:.4f}")

Prediction: Varicose veins, Probability: 0.2590
Prediction: Diabetes , Probability: 0.0799
Prediction: Hypothyroidism, Probability: 0.0671
Prediction: Hyperthyroidism, Probability: 0.0464
Prediction: Hepatitis C, Probability: 0.0391
